In [1]:
from scipy.stats import poisson
import matplotlib.pyplot as plt
# import pyodbc
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
FinalData = pd.DataFrame([])
foo = pd.DataFrame([])
tup_now = datetime.now().timetuple()
dateToday =datetime.today()
today = datetime.strftime(dateToday,'%Y-%m-%d')

In [3]:
path = "//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/Poisson/sql_output/InnerEventTimes_Data.csv"
data1 = pd.read_csv(path)
data1 = data1.sort_values(by=['CustomerAddressKey', 'Date2'], ascending=False)
data1['Date1'] = pd.to_datetime(data1['Date1'])
data1['Date2'] = pd.to_datetime(data1['Date2'])

In [4]:
AllButLastData1 = data1[data1.Date2 < today]
CustGroupABL = AllButLastData1.groupby('CustomerAddressKey').mean().drop(['Row1','Daily_Revenue'],axis=1)
FinalData = data1.merge(
    CustGroupABL, on='CustomerAddressKey').rename(
    index=str, columns={'InnerEventTime_y':'AvgInnerEventTime','InnerEventTime_x':'DaySinceLastPurchase'}).drop('Row1',axis=1)

In [7]:
FinalData = FinalData.assign(p = 0).rename(index=str, columns={'p':'Probability'})

In [8]:
FinalData.Probability = (poisson.cdf(FinalData.DaySinceLastPurchase, FinalData.AvgInnerEventTime, loc=0))

In [9]:
path = "//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/Poisson/"
writeFileName = path + 'innerEventTime_Current.csv'
FinalData.to_csv(writeFileName, index=False)
writeFilename_archive = path + 'csv_archive/' + str(tup_now.tm_year) + '_' + str(tup_now.tm_mon).zfill(2) \
                        + '_' + str(tup_now.tm_mday).zfill(2) + '_' + 'innerEventTime_' \
                        + str(tup_now.tm_hour).zfill(2) + str(tup_now.tm_min).zfill(2) + str(tup_now.tm_sec).zfill(2) \
                        + '.csv'
FinalData.to_csv(writeFilename_archive, index=False)

In [10]:
FinalData['Date2'] = pd.to_datetime(FinalData['Date2'])
FinalData['Date1'] = pd.to_datetime(FinalData['Date1'])

In [11]:
MostRecentEvent = FinalData.sort_values(by='Date2').groupby('CustomerAddressKey').last()
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [12]:
path = "//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/Poisson/"
writeFileName = path + 'lastInnerEventTime_Current.csv'
MostRecentEvent.to_csv(writeFileName, index=True)
writeFilename_archive = path + 'csv_archive/' + str(tup_now.tm_year) + '_' + str(tup_now.tm_mon).zfill(2) \
                        + '_' + str(tup_now.tm_mday).zfill(2) + '_' + 'lastInnerEventTime_' \
                        + str(tup_now.tm_hour).zfill(2) + str(tup_now.tm_min).zfill(2) + str(tup_now.tm_sec).zfill(2) \
                        + '.csv'
MostRecentEvent.to_csv(writeFilename_archive, index=True)